## RAG Agent with Multi Data Source Implementation LLMs.

In [ ]:
import os
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()

# environment variables call
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Langchain Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
## WifiPedia Tool

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki_tool

In [ ]:
## Arxiv Tool

from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_api_wapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wapper)
arxiv_tool

In [ ]:
## Own Custom Tool

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

loader = WebBaseLoader(os.getenv("WEB_BASE_URL"))
docs = loader.load()

# Split documents into chunks
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

# Use HuggingFaceEmbeddings for vectorization
vectorstore = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

# Create a retriever from the vectorstore
retriever = vectorstore.as_retriever()
retriever

In [ ]:
# Create a custom tool for the retriever

from langchain.tools.retriever import create_retriever_tool

custom_tool = create_retriever_tool(
    retriever=retriever, # This is the retriever we created as Document Retriever
    name="langchain_search", # This is the name of the tool
    description="Search information about Langchain" # This is the description of the tool
)
custom_tool

In [ ]:
# Importing the tools into the agent

tools = [wiki_tool, custom_tool, arxiv_tool]
tools

In [ ]:
## Importing the tools into the agent 

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0
)
llm

In [ ]:
## Prompt Template
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent") # Use of hub to pull the prompt template
prompt.messages


In [ ]:
## Agents
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)
agent

In [ ]:
# Create a retrieval chain with the retriever and agent

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(
    retriever,
    agent
)
retrieval_chain

In [ ]:
## To run Agent we need Agent Executor

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=retrieval_chain,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True
)
agent_executor

In [ ]:
## Run the agent executor with a query
query = "What is Langchain? What are its use cases?"
response = agent_executor.invoke({"input": query})
print(response['answer'])

In [ ]:
response1 = agent_executor.invoke({"input": "What's the paper 1605.08386 about"})
response1["answer"]